In [ ]:
# ============================
# Result Summary w/ Relative Drops (Teacher vs LoRA adapters)
# Paste & run as one cell in Colab
# ============================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install -q transformers datasets evaluate accelerate peft openpyxl

import os, json, math, pprint
import numpy as np, pandas as pd, torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from peft import PeftModel
import evaluate

# ---------- USER SETTINGS (edit if needed) ----------
DRIVE_BASE = "/content/drive/MyDrive/Colab Notebooks/HindiCodeMix"
DRIVE_BASE_LoRA = "/content/drive/MyDrive/Colab Notebooks/LoRA/HindiCodeMix"
SPLIT_BASE = "/content/drive/MyDrive/Colab Notebooks/HindiCodeMix/data_processed"
TEACHER_DIR = os.path.join(DRIVE_BASE, "results_teacher_4epoch", "model")
TOKENIZER_DIR = os.path.join(os.path.dirname(TEACHER_DIR), "tokenizer")
TEST_CSV = os.path.join(SPLIT_BASE, "test.csv")
ADAPTERS_ROOT = "/content/drive/MyDrive/Colab Notebooks/LoRA/HindiCodeMix/DropOut_0.05"  # folder containing adapter subfolders

OUT_CSV = os.path.join(DRIVE_BASE_LoRA, "lora_result_summary_with_drops_out0.05.csv")
OUT_XLSX = os.path.join(DRIVE_BASE_LoRA, "lora_result_summary_with_drops_out0.05.xlsx")
OUT_PLOT = os.path.join(DRIVE_BASE_LoRA, "lora_results_plots_with_drops_out0.05.png")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_EVAL = 32
MAX_LEN = 64

print("DEVICE:", DEVICE)

# ---------- Sanity checks ----------
if not os.path.exists(TEST_CSV):
    raise FileNotFoundError(f"Test CSV not found at {TEST_CSV}")

test_df = pd.read_csv(TEST_CSV)
if "review" not in test_df.columns or "label" not in test_df.columns:
    raise RuntimeError("test.csv must contain columns 'review' and 'label'.")

texts = test_df["review"].astype(str).tolist()
labels = test_df["label"].astype(int).tolist()

# ---------- Load tokenizer ----------
try:
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_DIR, use_fast=True, local_files_only=True)
    print("Loaded tokenizer from:", TOKENIZER_DIR)
except Exception as e:
    print("Local tokenizer load failed:", e, "\nFalling back to distilbert-base-multilingual-cased")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased", use_fast=True)

if tokenizer.pad_token is None:
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        tokenizer.add_special_tokens({"pad_token": "[PAD]"})

enc = tokenizer(texts, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")
indices = list(range(len(texts)))
batches = [indices[i:i+BATCH_EVAL] for i in range(0, len(indices), BATCH_EVAL)]

# ---------- Metrics helpers ----------
acc_metric = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")

def eval_model_logits(model, device):
    model.to(device).eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_idx in tqdm(batches, desc="Eval batches"):
            inp = { "input_ids": enc["input_ids"][batch_idx].to(device),
                    "attention_mask": enc["attention_mask"][batch_idx].to(device) }
            if "token_type_ids" in enc:
                inp["token_type_ids"] = enc["token_type_ids"][batch_idx].to(device)
            out = model(**inp)
            logits = getattr(out, "logits", out[0] if isinstance(out, (list,tuple)) else out)
            if hasattr(logits, "detach"):
                preds = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
            else:
                preds = np.argmax(np.asarray(logits), axis=-1).tolist()
            all_preds.extend(preds)
            all_labels.extend([labels[i] for i in batch_idx])
    acc = float(acc_metric.compute(predictions=all_preds, references=all_labels)["accuracy"])
    f1  = float(f1_metric.compute(predictions=all_preds, references=all_labels, average="macro")["f1"])
    return acc, f1

def count_total_params(model):
    return sum(p.numel() for p in model.parameters())

def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def folder_on_disk_size_bytes(path):
    total = 0
    for root, _, files in os.walk(path):
        for f in files:
            try:
                total += os.path.getsize(os.path.join(root, f))
            except Exception:
                pass
    return total

# ---------- Load teacher ----------
if not os.path.isdir(TEACHER_DIR):
    raise RuntimeError(f"Teacher model not found at {TEACHER_DIR}")
print("Loading teacher from", TEACHER_DIR)
teacher = AutoModelForSequenceClassification.from_pretrained(TEACHER_DIR, local_files_only=True, output_attentions=True, output_hidden_states=True)
teacher_acc, teacher_f1 = eval_model_logits(teacher, DEVICE)
teacher_total_params = count_total_params(teacher)
teacher_disk_bytes = folder_on_disk_size_bytes(TEACHER_DIR)
print(f"Teacher -> Acc: {teacher_acc:.4f}, Macro-F1: {teacher_f1:.4f}, Params: {teacher_total_params:,}")

records = []
records.append({
    "Label": "Teacher",
    "Accuracy": teacher_acc,
    "Macro-F1": teacher_f1,
    "TotalParams": teacher_total_params,
    "TrainableParams": teacher_total_params,
    "DiskBytes": teacher_disk_bytes,
    "ParamReduction%": None,
    "RelAccDrop%": None,
    "RelF1Drop%": None,
    "Path": TEACHER_DIR,
    "Notes": "teacher"
})

teacher.to("cpu"); del teacher; torch.cuda.empty_cache()

# ---------- Discover adapters ----------
adapter_dirs = []
if os.path.isdir(ADAPTERS_ROOT):
    for name in sorted(os.listdir(ADAPTERS_ROOT)):
        full = os.path.join(ADAPTERS_ROOT, name)
        if os.path.isdir(full):
            # heuristic: directory with adapter artifacts or metadata
            files = os.listdir(full)
            markers = {"adapter_config.json","pytorch_model.bin","adapter_results.json","pytorch_model.bin.index.json"}
            if markers.intersection(files) or any(f.startswith("pytorch_model") for f in files):
                adapter_dirs.append(full)
            else:
                # search nested
                for root, _, fns in os.walk(full):
                    if markers.intersection(fns):
                        adapter_dirs.append(full); break
else:
    print("ADAPTERS_ROOT not found:", ADAPTERS_ROOT)

print("Detected adapters:", adapter_dirs)

# ---------- Evaluate adapters ----------
for adp in adapter_dirs:
    print("\nProcessing adapter:", adp)
    meta = {}
    meta_path = os.path.join(adp, "adapter_results.json")
    if os.path.exists(meta_path):
        try:
            meta = json.load(open(meta_path, "r"))
        except Exception:
            meta = {}

    # try to load as PEFT adapter (preferred)
    try:
        base_for_eval = AutoModelForSequenceClassification.from_pretrained(TEACHER_DIR, local_files_only=True, config=AutoConfig.from_pretrained(TEACHER_DIR))
    except Exception:
        base_for_eval = AutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=2)

    try:
        peft_loaded = PeftModel.from_pretrained(base_for_eval, adp, is_trainable=False)
        peft_loaded.to(DEVICE)
        acc, f1 = eval_model_logits(peft_loaded, DEVICE)
        total_p = count_total_params(peft_loaded)
        train_p = count_trainable_params(peft_loaded)
        disk_b = folder_on_disk_size_bytes(adp)

        rel_acc = round(100.0 * (teacher_acc - acc) / teacher_acc, 4) if teacher_acc else None
        rel_f1  = round(100.0 * (teacher_f1 - f1) / teacher_f1, 4) if teacher_f1 else None
        param_red = round(100.0 * (1.0 - (train_p / teacher_total_params)), 4)

        records.append({
            "Label": os.path.basename(adp),
            "Accuracy": round(acc, 6),
            "Macro-F1": round(f1, 6),
            "TotalParams": total_p,
            "TrainableParams": train_p,
            "DiskBytes": disk_b,
            "ParamReduction%": param_red,
            "RelAccDrop%": rel_acc,
            "RelF1Drop%": rel_f1,
            "Path": adp,
            "Notes": meta.get("notes", "peft_adapter")
        })

        peft_loaded.to("cpu"); del peft_loaded; del base_for_eval; torch.cuda.empty_cache()

    except Exception as e:
        print("PEFT load failed for", adp, "— trying to load as full model:", e)
        try:
            model_full = AutoModelForSequenceClassification.from_pretrained(adp, local_files_only=True, output_attentions=True, output_hidden_states=True)
            acc, f1 = eval_model_logits(model_full, DEVICE)
            total_p = count_total_params(model_full)
            train_p = count_trainable_params(model_full)
            disk_b = folder_on_disk_size_bytes(adp)

            rel_acc = round(100.0 * (teacher_acc - acc) / teacher_acc, 4) if teacher_acc else None
            rel_f1  = round(100.0 * (teacher_f1 - f1) / teacher_f1, 4) if teacher_f1 else None
            param_red = round(100.0 * (1.0 - (train_p / teacher_total_params)), 4)

            records.append({
                "Label": os.path.basename(adp),
                "Accuracy": round(acc,6),
                "Macro-F1": round(f1,6),
                "TotalParams": total_p,
                "TrainableParams": train_p,
                "DiskBytes": disk_b,
                "ParamReduction%": param_red,
                "RelAccDrop%": rel_acc,
                "RelF1Drop%": rel_f1,
                "Path": adp,
                "Notes": "full-model"
            })

            model_full.to("cpu"); del model_full; torch.cuda.empty_cache()
        except Exception as e2:
            print("Failed to load adapter or full model for", adp, "=>", e2)
            continue

# ---------- Build dataframe & format ----------
df = pd.DataFrame(records)
if df.empty:
    print("No records found.")
else:
    # numeric formatting
    df["Accuracy"] = df["Accuracy"].astype(float).round(6)
    df["Macro-F1"] = df["Macro-F1"].astype(float).round(6)
    df["Params(M)"] = (df["TotalParams"].astype(float) / 1e6).round(4)
    df["AdapterParams(K)"] = (df["TrainableParams"].astype(float) / 1e3).round(3)
    df["Disk(MB)"] = (df["DiskBytes"].astype(float) / (1024*1024)).round(4)
    df["ParamReduction%"] = df["ParamReduction%"].astype(float).round(4)
    # ensure rel drop cols exist
    if "RelAccDrop%" not in df.columns: df["RelAccDrop%"] = None
    if "RelF1Drop%" not in df.columns: df["RelF1Drop%"] = None
    df["RelAccDrop%"] = df["RelAccDrop%"].astype(float).round(4)
    df["RelF1Drop%"] = df["RelF1Drop%"].astype(float).round(4)

    # order: teacher first, then adapters sorted by Macro-F1 desc
    teacher_row = df[df["Label"]=="Teacher"]
    others = df[df["Label"]!="Teacher"].sort_values(by="Macro-F1", ascending=False)
    df_out = pd.concat([teacher_row, others], ignore_index=True)

    cols_present = ["Label","Accuracy","Macro-F1","Params(M)","AdapterParams(K)","ParamReduction%","RelAccDrop%","RelF1Drop%","Disk(MB)","Path","Notes"]
    cols_present = [c for c in cols_present if c in df_out.columns]
    df_final = df_out[cols_present].reset_index(drop=True)

    df_final.to_csv(OUT_CSV, index=False)
    try:
        df_final.to_excel(OUT_XLSX, index=False)
    except Exception as e:
        print("Excel save failed:", e)

    print("\nSaved summary to:", OUT_CSV, OUT_XLSX)
    display(df_final)

    # ---------- Plots ----------
    if not df_final.empty:
        plt.figure(figsize=(12,5))
        plt.subplot(1,2,1)
        plt.scatter(df_final["Params(M)"], df_final["Accuracy"])
        for i,row in df_final.iterrows():
            plt.annotate(row["Label"], (row["Params(M)"], row["Accuracy"]))
        plt.xlabel("Params (M)")
        plt.ylabel("Accuracy")
        plt.title("Accuracy vs Model Size")

        plt.subplot(1,2,2)
        x = df_final["ParamReduction%"].fillna(0)
        y = df_final["Macro-F1"]
        plt.scatter(x, y)
        for i,row in df_final.iterrows():
            plt.annotate(row["Label"], (row["ParamReduction%"], row["Macro-F1"]))
        plt.xlabel("ParamReduction%")
        plt.ylabel("Macro-F1")
        plt.title("ParamReduction% vs Macro-F1")

        plt.tight_layout()
        plt.savefig(OUT_PLOT, dpi=200)
        print("Saved plots to:", OUT_PLOT)
        display(plt.show())

print("Done.")
